<center>
<span style="font-size: 36px;">Youtube Keywords & Analysis</span>
</center>

<center>
<span style="font-size: 26px;">Tools</span>
</center>

In [ ]:
import pandas as pd
import re
import ast
import statistics as st
from collections import Counter, defaultdict

<center>
<span style="font-size: 26px;">Function for data cleansing</span>
</center>

In [ ]:

def load_and_clean_csv(path):
    df = pd.read_csv(path)
    df_sorted = df.sort_values(by='Date', ascending=False)
    return df_sorted.drop_duplicates(subset='Video ID', keep='first').reset_index(drop=True)


# 데이터 클렌징 함수 정의
def clean_data(df):
    df = df.copy()  # Copy to avoid SettingWithCopyWarning
    df['Channel Keywords'] = df['Channel Keywords'].fillna('').str.replace(r"[\[\]'#]", '', regex=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df['Uploaded Date'] = pd.to_datetime(df['Uploaded Date'])
    df['Trend Date'] = ((df['Date'] - df['Uploaded Date']).dt.days).replace(0, 1)
    df['Tags'] = df['Tags'].str.replace(r"[^\w,\s]", '', regex=True).str.lower().fillna('')
    df = df[df['Channel Title'] != "MrBeast"]  
    return df

# Outlier removal
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Fill NaN
def fill_nan_values(df, column, ratio_column):
    return df[column].fillna(df['Views'] * ratio_column).astype(int)

# Decreasing gap between data
def normalize_columns(df, columns):
    for col in columns:
        df[f'Norm{col}'] = (df[col] ** (1/2)).astype(int) / (df[col] ** (1/2)).astype(int).max()
    return df

# TagScore & Channel Score calculation
def calculate_scores(df):
    df['TagScore'] = (df['NormViews'] * 0.6) + (df['NormEngagement'] * 0.2) + (df['NormTrend Date'] * 0.2)
    df['Channel Score'] = (df['NormChannel Total View'] * 0.7) + (df['NormChannel Subscribers'] * 0.3)
    df['Total Score'] = (df['TagScore'] * 0.7) + (df['Channel Score'] * 0.3)
    return df

# Creating calculated data table
def process_data(df):
    df = clean_data(df)

    # Remove outliers from likes & comments
    df_clean_likes = remove_outliers(df.dropna(subset=['Likes']), 'Likes')
    df_clean_comments = remove_outliers(df.dropna(subset=['Comments']), 'Comments')

    # fill NaN value by like & comment to view ratio
    like_to_view_ratio = df_clean_likes['Likes'].sum() / df_clean_likes['Views'].sum()
    comment_to_view_ratio = df_clean_comments['Comments'].sum() / df_clean_comments['Views'].sum()
    df.loc[:, 'Likes'] = fill_nan_values(df, 'Likes', like_to_view_ratio)
    df.loc[:, 'Comments'] = fill_nan_values(df, 'Comments', comment_to_view_ratio)
    df.loc[:, 'Engagement'] = (df['Likes'] + df['Comments']) / df['Views']
    df.loc[:, 'NormTrend Date'] = df['Trend Date'] / df['Trend Date'].max()
    df.loc[:, 'NormEngagement'] = df['Engagement'] / df['Engagement'].max()

    # Decrease big values
    columns_to_transform = ['Views', 'Likes', 'Comments', 'Channel Total View', 'Channel Subscribers']
    df = normalize_columns(df, columns_to_transform)

    # TagScore / Channel Score calculation
    df = calculate_scores(df)

    return df


In [ ]:
krcsv = load_and_clean_csv("kr2024(7-10).csv")
jpcsv = load_and_clean_csv("jp2024(7-10).csv")

krcsv = process_data(krcsv)
jpcsv = process_data(jpcsv)

<center>
<span style="font-size: 26px;">Function for database</span>
</center>

In [1]:
# Category table
def create_category_table(df, country_id, start_id):
    category_table = []
    for category_id, category in enumerate(df['Category'].unique(), start=start_id):
        category_table.append({
            'Country_id': country_id,
            'Category': category,
            'Category_id': category_id
        })
    return pd.DataFrame(category_table)


# Video information table
def create_video_info_table(df, category_table):
    merged_df = pd.merge(df, category_table, on='Category', how='left')
    return merged_df[['Title', 'Category_id', 'Views', 'Trend Date', 'Channel Title','TagScore']].sort_values(by='Views').reset_index(drop=True)

# Tag information table
def create_tag_table(df,categorytable):
    # Number of each tags
    tag_count = Counter()
    tag_category = defaultdict(list)
    for tags, category in zip(df['Tags'], df['Category']):
        for tag in tags.split(', '):
            if tag:
                tag_count[tag] += 1
                tag_category[tag].append(category)

    most_frequent_categories = {tag: Counter(categories).most_common(1)[0][0] for tag, categories in tag_category.items()}

    # Create tag table
    tag_table = pd.DataFrame({
        'Tag': list(tag_count.keys()),
        'Count': list(tag_count.values()),
        'Category': [most_frequent_categories[tag] for tag in tag_count.keys()]
    })
    tag_table = pd.merge(tag_table,categorytable,on='Category',how='left')

    # Calculate tag score
    tag_scores = defaultdict(float)
    for tags, total_score in zip(df['Tags'], df['Total Score']):
        for tag in tags.split(', '):
            if tag:
                tag_scores[tag] += total_score

    tag_scores_df = pd.DataFrame.from_dict(tag_scores, orient='index', columns=['Score']).reset_index()
    tag_scores_df.rename(columns={'index': 'Tag'}, inplace=True)

    # Final tag table
    tag_table_final = pd.merge(tag_table, tag_scores_df, on='Tag', how='left').sort_values(by='Score', ascending=False).reset_index(drop=True)
    tag_table_final['Score'] = tag_table_final['Score'] ** (1/2)
    return tag_table_final[['Tag','Category_id','Count','Score']]


In [ ]:
# Country information table
c_info = {'Country': ['KR', 'JP'], 'Country_id': [1, 2]}
country_df = pd.DataFrame(data=c_info)

kr_category_table = create_category_table(krcsv, 1, 101)
jp_category_table = create_category_table(jpcsv, 2, 201)

kr_category_merge2 = create_video_info_table(krcsv, kr_category_table)
jp_category_merge2 = create_video_info_table(jpcsv, jp_category_table)

kr_tag_table_final = create_tag_table(krcsv,kr_category_table)
jp_tag_table_final = create_tag_table(jpcsv,jp_category_table)